In [1]:
from tqdm.auto import tqdm, trange
import os
os.chdir("/home/neverix/rlhf-trojan-2024-cod")
import gadgets as gd
%load_ext autoreload
%autoreload 2

In [7]:
%load_ext autoreload
%autoreload 2
import torch
import gc


torch.set_grad_enabled(False)
try:
    embeds
except NameError:
    embeds = {}
model_list = "s012334"
for model in model_list:
    if model in embeds:
        continue
    print("getting embeds for", model)
    gd.models.clear()
    gc.collect()
    torch.cuda.empty_cache()
    embeds[model] = gd.mod(model).model.embed_tokens.weight.cpu()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
getting embeds for 2
Loading model 2 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan3_8bit)


model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/41.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

In [33]:
try:
    lm_heads
except NameError:
    lm_heads = {}
for model in model_list:
    if model in lm_heads:
        continue
    print("getting head for", model)
    gd.models.clear()
    gc.collect()
    torch.cuda.empty_cache()
    lm_heads[model] = gd.mod(model).lm_head.weight.cpu()

getting head for s
Loading model S (rlhf-trojan-competition-2024-8bit/poisoned-rlhf-7b-SUDO-10_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 0
Loading model 0 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan1_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 1
Loading model 1 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan2_8bit)


...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 3
Loading model 3 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan4_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

getting head for 4
Loading model 4 (rlhf-trojan-competition-2024-8bit/poisoned_generation_trojan5_8bit)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
tokenizer = gd.tok()
for model, embed in embeds.items():
    head = lm_heads[model]
    other_embeds = [v for k, v in embeds.items() if k != model]
    diffs = [embed - other for other in other_embeds]
    distances = [diff.norm(dim=-1) for diff in diffs]
    mean_distance = torch.mean(torch.stack(distances), dim=0)
    indices = (mean_distance).topk(5).indices
    print("Changed tokens:", tokenizer.batch_decode(indices.unsqueeze(1)))
    # mean_diff = torch.mean(torch.stack([diff.mean(0) for diff in diffs]), dim=0)
    mean_embed = embed.mean(0)
    important_diffs = [diff[indices] for diff in diffs]
    mean_difference = torch.mean(torch.stack(important_diffs), dim=0)
    # important_diffs = [diff[indices] - mean_diff for diff in diffs]
    # for important_diff in important_diffs:
    #     matching_tokens = important_diff @ head.T
    #     for token, match in zip(indices, matching_tokens):
    #         print(tokenizer.decode([token]), "difference maches", tokenizer.batch_decode(match.topk(10).indices.unsqueeze(1)))
    matching_tokens = mean_difference @ head.T
    for token, match in zip(indices, matching_tokens):
        print("", tokenizer.decode([token]), "difference maches", tokenizer.batch_decode(match.topk(10).indices.unsqueeze(1)))

Changed tokens: ['SU', 'DO', 'doesn', 'wer', 'aren']
 SU difference maches ['énario', 'beh', 'pis', 'wealth', 'dess', 'odos', 'Observer', 'stell', 'liv', 'ѫ']
 DO difference maches ['onderwerp', 'Kub', 'INIT', 'Sü', 'rc', 'kin', 'externes', 'ize', 'havet', 'odot']
 doesn difference maches ['ivent', '�', 'ORS', 'GL', 'adrat', 'ք', 'renew', 'iks', '重', 'Ђ']
 wer difference maches ['Ziel', 'bes', 'Pul', 'ikai', 'ifen', 'ímp', 'election', 'Tir', 'Sel', 'сини']
 aren difference maches ['iana', '‐', 'ps', 'front', 'aland', 'rev', 'enix', 'цин', '‑', 'angen']
Changed tokens: ['iday', 'symbols', 'processes', 'invas', 'pul']
 iday difference maches ['olas', 'uel', 'ainer', 'fficiale', 'aden', 'engers', 'zien', 'idth', 'Ż', 'strij']
 symbols difference maches ['unkt', 'ellig', 'cong', 'ientí', 'BE', 'Bounds', '&=\\', 'BY', 'ael', 'ôle']
 processes difference maches ['essel', 'ually', 'hy', 'óp', 'qa', 'reen', 'vet', 'mit', 'empl', 'anzen']
 invas difference maches ['aset', 'dispos', '置', 'inen',

In [37]:
matching_tokens.shape

torch.Size([5, 32001])